In [1]:
import requests
import time
import re
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import http.cookiejar
import os 
import sys
import ijson
import io
from bson.code import Code
import json
import string
from datetime import datetime
from nltk.corpus import stopwords
from datetime import datetime
import re 
import matplotlib.pyplot as plt
from textblob import TextBlob
from datetime import datetime
import re 
import pandas as pd
from pandas import DataFrame
import random
import numpy as np

### 1.Scrape Nasdaq to get all the news link, title and dynamic api link

In [3]:
#The website url does not get the data by get request, it calls an api which can be found by inspecting
headers = {
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-Encoding":"gzip, deflate",
"Accept-Language":"en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
"Connection":"keep-alive",
"Host":"www.nasdaq.com",
"Referer":"http://www.nasdaq.com",
"Upgrade-Insecure-Requests":"1",
'User-agent': 'Mozilla/5.0'} 

result_links=[]
result_titles=[]
for pg in range(1,2): #page number, can be changed 
    api_url = 'https://www.nasdaq.com/api/v1/search?q=tesla&offset='+str((pg-1)*10)  # get dynamic api link
    response=requests.get(api_url,headers=headers)
    json_result = json.loads(response.content) #the result is a json file 
    html_result=json_result['items'] #parse the json file 
    no_of_result=0 #count number of result 
    for el in html_result:
        time.sleep(3) 
        soup=BeautifulSoup(el,'html.parser')  ##parse the html beautifully
        no_of_result=no_of_result+1
        #print(no_of_result," results") ##to help debugging
        
        #find each search result title,link, and timestamp
        result_title_tag=soup.select("h2.search-result__title>a")[0] 
        result_title=result_title_tag.get("title")
        partial_link=result_title_tag.get("href") #link is only partial
        try:
            date_stamp_string=re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2}',partial_link)[0]
            date_stamp=datetime.strptime(date_stamp_string, '%Y-%m-%d') #transfer to date object
            date_stamp_bchmark=datetime.strptime('2019-10-01','%Y-%m-%d') #2019.10.01 is the time stock price start to soar
        except:
            continue
        
        #skip the symbol results which are not news
        result_eyebrow=soup.select("div.search-result__eyebrow")[0]
        #skip the ones that are not news or the ones that are earlier than 2019.10.01
        if result_eyebrow.text!="Symbols" and date_stamp>=date_stamp_bchmark: 
            full_link="https://www.nasdaq.com"+str(partial_link)
            result_links.append(full_link)
            result_titles.append(result_title)
            print(result_title)
            print(full_link)
    if response.status_code!=200:
        raise ValueError("Invalid Response Received From Webserver")
    #print(result)

Tesla, Tesla, Tesla: Where Does The Rally End?
https://www.nasdaq.com/articles/tesla-tesla-tesla%3A-where-does-the-rally-end-2020-02-06
Tesla Collar Update
https://www.nasdaq.com/articles/tesla-collar-update-2019-12-26


## 2. download the links 

In [4]:
#need to get the cookie 
session_requests = requests.session()
# going to the home page while logged in  
r2=session_requests.get('https://www.nasdaq.com/',headers=headers)
cookie=r2.cookies.get_dict()
print("cookie is :",cookie)

for link in result_links:
    ##download each html  GET requests
    response=requests.get(link,cookies=cookie,headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser') 
#   print(soup)
#   write file
    no_of_news=result_links.index(link)
    title=str(result_titles[no_of_news]).translate(str.maketrans(' ','_',string.punctuation))
    with open("tesla_news_"+str(no_of_news+1)+"_"+str(title)+".htm","w",encoding='utf-8') as file:
        html_unicode=UnicodeDammit(str(soup)).unicode_markup
        file.write(html_unicode)

cookie is : {'AKA_A2': 'A', 'ak_bmsc': 'F9BFB1E60EA85BA42708FF2BE1E012BA173DC397B96F000018E0655E50CF175F~plm1qGiMa59El3H8bqlRNUJsjZ/YpT04lumh2xvRzy7YejE5YOdkvxL7vqZ1p8gF5ADLjUUn5Fb7qfIuWFumPxT0MMx2FqIr15fn06O+P3vJ7hEQirIbRXTVix9X4SRyRzXJaZBiWuFT78qCAIR8aCpNnybVgM0h7TmUdpgX5fe7kR3SlIt6ny8R+Jdbnr6+mZeX5zR0QOeIHbH6fI7KBzxZZQmKqDfjVcyK4oBlTAJrs='}


## 3. Parse and get the content of the news 

In [372]:
#a typical leftover thing : [<time class="timestamp__date" datetime="">Feb 13, 2020 7:47AM EST</time>]

[<time class="timestamp__date" datetime="">Feb 13, 2020 7:47AM EST</time>]


In [2]:
paragraph_list=[]
date_list=[]
directory = os.getcwd() #get the directory
for filename in os.listdir(directory):
    paragraph=""
    if filename.endswith('.htm') == False:
        continue
    try:
        with open(os.path.join(directory, filename), 'r',encoding="utf-8") as file:
            text = file.read()
            soup = BeautifulSoup(text, 'html.parser') 
            #print(soup)
            #get paragraph
            paragraphs = soup.findAll('p') 
        if not paragraphs:
            paragraph=None
        else:
            #print(filename)
            try:
                datestr=soup.select('time[datetime]')[0]['datetime']
                #datestamp
                clean_date=datestr[0:datestr.rfind('-')] #strip out the clean date time
                datetime_object = datetime.strptime(clean_date, '%Y-%m-%dT%H:%M:%S')
                date_list.append(datetime_object.date()) # get only the date
                #print(datetime_object.date())
                for el in paragraphs[1:]: #first <p> label is meaningless
                    if el.text=="The views and opinions expressed herein are the views and opinions of the author and do not necessarily reflect those of Nasdaq, Inc.":
                        break
                    paragraph=paragraph+el.text
                    #print(el.text) 
                paragraph_list.append(paragraph)
            except:
                #Some news doesn't have value for datetime tag 
                datestr=""
               
    except:
        print("file "+filename,sys.exc_info()[0])

In [3]:
##There are 814 html file but some are skipped bcs they don't have datetime stamp 
##To see if paragraph and date has the same number of items
print('There are ',len(paragraph_list),'paragraphs and',len(date_list),'date stamps')

There are  806 paragraphs and 806 date stamps


In [61]:
sums=[]
date_sentim_pair=dict()
for pa in paragraph_list:
    sum=0
    blob=TextBlob(pa)
    for sentence in blob.sentences:
        sentim=sentence.sentiment.polarity
        sum=sum+sentim
    i=paragraph_list.index(pa)
    sentim_avg=sum/len(blob.sentences)
    print(date_list[i])
    print(sentim_avg)
    #average sentiment 
    date_sentim_pair[date_list[i]]=sentim_avg

2019-10-07
0.1498438056978621
2019-12-29
-0.02905844155844156
2019-10-23
0.02952482375462172
2020-02-06
0.201583167989418
2020-02-20
0.021428571428571432
2019-11-22
0.2023306477473144
2020-02-21
0.31813124375624374
2020-02-06
-0.05962962962962962
2019-10-02
0.04348169191919192
2019-10-23
0.02638037117203784
2019-10-24
0.08117283950617284
2019-11-21
0.05916666666666667
2019-10-24
0.05494447021232735
2019-10-03
-0.11574074074074076
2019-11-05
0.11325757575757577
2020-01-30
0.2825
2019-11-26
0.09165896804483761
2019-12-27
0.05652830387205388
2020-02-04
-0.019722222222222224
2020-01-08
0.18427272727272728
2020-02-03
0.039679487179487175
2020-02-10
0.26792478354978355
2019-11-22
0.10991961279461278
2020-01-17
0.18622804232804238
2020-02-07
-0.01636679292929293
2020-01-30
0.05105863833136559
2020-02-25
0.10920138625182484
2020-01-31
0.19494017556517557
2020-02-02
0.26666666666666666
2019-10-14
0.051112125721500715
2019-10-04
0.005402146464646462
2020-02-09
0.14932414204153333
2019-11-04
-0.0

In [685]:
date_parag_pair=dict()
for pa in paragraph_list:
    i=paragraph_list.index(pa)
    date_parag_pair[date_list[i]]=pa
#print(random.choice(list(date_parag_pair.items())))
print(random.sample(date_parag_pair.items(),1)) #choose a random sample to check

[(datetime.date(2020, 1, 24), "As of late, it has definitely been a great time to be an investor in Tesla, Inc. TSLA. The stock has moved higher by 80% in the past month, while it is also above its 20 Day SMA too. This combination of strong price performance and favorable technical, could suggest that the stock may be on the right path.We certainly think that this might be the case, particularly if you consider TSLA’s recent earnings estimate revision activity. From this look, the company’s future is quite favorable; as TSLA has earned itself a Zacks Rank #2 (Buy), meaning that its recent run may continue for a bit longer, and that this isn’t the top for the in-focus company. You can see the complete list of today’s Zacks #1 Rank stocks here.Zacks Top 10 Stocks for 2020In addition to the stocks discussed above, would you like to know about our 10 finest buy-and-hold tickers for the entirety of 2020?Last year's 2019 Zacks Top 10 Stocks portfolio returned gains as high as +102.7%. Now a 